# 加载训练模型实验

## 数据准备与加载

In [1]:
import joblib
import math
import copy
import time
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model 
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, explained_variance_score, r2_score, mean_squared_error

In [2]:
# 加载训练数据
def load_data(path):
    df = pd.read_csv(path)
    dv = df.values
    X = dv[:, 1:-2]
    Y = dv[:, -2:]
    return X[:, :, np.newaxis], Y

# 数据准备
X, y = load_data('./dataset/location_data.csv')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2021)

X02, y02 = load_data('./dataset/location_data02.csv')

# 针对b_gru处理数据
X_train_2 = copy.deepcopy(X_train)
X_test_2 = copy.deepcopy(X_test)
X02_2 = copy.deepcopy(X02)
for i in range(len(X_train_2)):
    for j in range(len(X_train_2[i])-1, 0, -1):
        X_train_2[i][j] = X_train_2[i][j] - X_train_2[i][j-1]
for i in range(len(X_test_2)):
    for j in range(len(X_test_2[i]) - 1, 0, -1):
        X_test_2[i][j] = X_test_2[i][j] - X_test_2[i][j-1]   
for i in range(len(X02_2)):
    for j in range(len(X02_2[i])-1, 0, -1):
        X02_2[i][j] = X02_2[i][j] - X02_2[i][j-1]

## 创建展示结果

In [28]:
# arr = np.zeros((10, 4), dtype=int)
# res = pd.DataFrame(arr, index = ['LSTM','LSTM_loss','SVM','RNN','RNN_loss','DNN','GRU','GRU_loss','B_GRU','B_GRU_loss'], 
#                    columns = ['MAE', 'RMSE','R2','TIME'])

arr = np.zeros((6, 4), dtype=int)
res = pd.DataFrame(arr, index = ['RNN', 'DNN', 'SVM', 'LSTM', 'GRU', 'B_GRU'], 
                   columns = ['MAE', 'RMSE', 'R2', 'TIME'])
res

,MAE,RMSE,R2,TIME
RNN,0,0,0,0
DNN,0,0,0,0
SVM,0,0,0,0
LSTM,0,0,0,0
GRU,0,0,0,0
B_GRU,0,0,0,0


## 导入双SVR模型计算指标

In [29]:
# 加载坐标模型
model_x = joblib.load('./models/SVR_x_2021.model')
model_y = joblib.load('./models/SVR_y_2021.model')

/Users/frank/anaconda3/envs/tensorflow/lib/python3.7/site-packages/sklearn/base.py:333: UserWarning: Trying to unpickle estimator SVR from version 0.23.2 when using version 1.0.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,


In [30]:
start = time.time()
# 预测结果分析
px = model_x.predict(X02.reshape(20, 100))
py = model_y.predict(X02.reshape(20, 100))
pxy = [[xi, yi] for xi, yi in zip(px, py)]
print(pxy[:10])
print('MAE指标:', mean_absolute_error(y02, pxy))
print('MSE指标:', mean_squared_error(y02, pxy))
print('RMSE指标:', math.sqrt(mean_squared_error(y02, pxy)))
print('r2指标:', r2_score(y02, pxy))
print('ev指标:', explained_variance_score(y02, pxy))
end = time.time()
print('time:', end-start)

[[2.019872536804463, 1.1504658854707914], [1.9174415171418377, 4.857493601728745], [2.8777523461834495, 8.218324483304807], [4.3622108033891775, 0.6274118267515414], [4.055571006636194, 6.1590862433955], [5.350674626615138, 2.8127786337651046], [6.42470271360842, 8.819694579139078], [6.980132678366614, 5.587903226497848], [8.356732356614094, 1.3613988644955959], [9.286397538342719, 7.910289796490444]]
MAE指标: 0.3727123714581049
MSE指标: 0.21134940529974042
RMSE指标: 0.45972753376292397
r2指标: 0.9740350753248724
ev指标: 0.9785745183118661
time: 0.01831674575805664


In [31]:
res.loc['SVM','MAE'] = mean_absolute_error(y02, pxy)
res.loc['SVM','RMSE'] = math.sqrt(mean_squared_error(y02, pxy))
res.loc['SVM','R2'] = r2_score(y02, pxy)
res.loc['SVM','TIME'] = end-start

## 导入LSTM模型计算指标

In [32]:
# 加载预训练模型
lstm = load_model('./models/LSTM_2021.h5')
lstm.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 100)               40800     
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 202       
Total params: 41,002
Trainable params: 41,002
Non-trainable params: 0
_________________________________________________________________


In [33]:
start = time.time()
# 预测测试集计算指标
pxy = lstm.predict(X_test)
pxy = np.squeeze(pxy)
print('MAE指标:', mean_absolute_error(y_test, pxy))
print('MSE指标:', mean_squared_error(y_test, pxy))
print('RMSE指标:', math.sqrt(mean_squared_error(y_test, pxy)))
print('r2指标:', r2_score(y_test, pxy))
print('ev指标:', explained_variance_score(y_test, pxy))
end = time.time()
print('time:', end-start)

MAE指标: 0.3008994502909482
MSE指标: 0.1800757818445658
RMSE指标: 0.4243533690741312
r2指标: 0.9750130417615185
ev指标: 0.9780773835064166
time: 0.5652210712432861


In [34]:
start = time.time()
# 02数据预测计算指标
pxy = lstm.predict(X02)
pxy = np.squeeze(pxy)
print('MAE指标:', mean_absolute_error(y02, pxy))
print('MSE指标:', mean_squared_error(y02, pxy))
print('RMSE指标:', math.sqrt(mean_squared_error(y02, pxy)))
print('r2指标:', r2_score(y02, pxy))
print('ev指标:', explained_variance_score(y02, pxy))
end = time.time()
print('time:', end-start)

MAE指标: 0.24164391636848448
MSE指标: 0.11536939369101676
RMSE指标: 0.33966070377807434
r2指标: 0.9859137922971934
ev指标: 0.9872240293856445
time: 0.06300115585327148


In [35]:
res.loc['LSTM','MAE'] = mean_absolute_error(y02, pxy)
res.loc['LSTM','RMSE'] = math.sqrt(mean_squared_error(y02, pxy))
res.loc['LSTM','R2'] = r2_score(y02, pxy)
res.loc['LSTM','TIME'] = end-start

## 导入RNN模型计算指标

In [36]:
# 加载预训练模型
rnn = load_model('./models/RNN_2021.h5')
rnn.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_1 (SimpleRNN)     (None, 100)               10200     
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 202       
Total params: 10,402
Trainable params: 10,402
Non-trainable params: 0
_________________________________________________________________


In [37]:
start = time.time()
# 预测测试集计算指标
pxy = rnn.predict(X_test)
pxy = np.squeeze(pxy)
print('MAE指标:', mean_absolute_error(y_test, pxy))
print('MSE指标:', mean_squared_error(y_test, pxy))
print('RMSE指标:', math.sqrt(mean_squared_error(y_test, pxy)))
print('r2指标:', r2_score(y_test, pxy))
print('ev指标:', explained_variance_score(y_test, pxy))
end = time.time()
print('time:', end-start)

MAE指标: 1.6656959114372745
MSE指标: 4.118840812627927
RMSE指标: 2.0294927476164895
r2指标: 0.4394924041787337
ev指标: 0.45112126522271057
time: 0.19918417930603027


In [38]:
start = time.time()
# 02数据预测计算指标
pxy = rnn.predict(X02)
pxy = np.squeeze(pxy)
print('MAE指标:', mean_absolute_error(y02, pxy))
print('MSE指标:', mean_squared_error(y02, pxy))
print('RMSE指标:', math.sqrt(mean_squared_error(y02, pxy)))
print('r2指标:', r2_score(y02, pxy))
print('ev指标:', explained_variance_score(y02, pxy))
end = time.time()
print('time:', end-start)

MAE指标: 1.7757722589969636
MSE指标: 4.573417597966725
RMSE指标: 2.1385550257046755
r2指标: 0.43664256207608587
ev指标: 0.46001657646560545
time: 0.052778005599975586


In [39]:
res.loc['RNN','MAE'] = mean_absolute_error(y02, pxy)
res.loc['RNN','RMSE'] = math.sqrt(mean_squared_error(y02, pxy))
res.loc['RNN','R2'] = r2_score(y02, pxy)
res.loc['RNN','TIME'] = end-start

## 导入DNN模型计算指标

In [40]:
# 加载预训练模型
dnn = load_model('./models/DNN_2021.h5')
dnn.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_4 (Dense)              (None, 10)                1010      
_________________________________________________________________
dense_5 (Dense)              (None, 2)                 22        
Total params: 11,132
Trainable params: 11,132
Non-trainable params: 0
_________________________________________________________________


In [41]:
start = time.time()
# 预测测试集计算指标
pxy = dnn.predict(X_test.reshape(400, 100))
pxy = np.squeeze(pxy)
print('MAE指标:', mean_absolute_error(y_test, pxy))
print('MSE指标:', mean_squared_error(y_test, pxy))
print('RMSE指标:', math.sqrt(mean_squared_error(y_test, pxy)))
print('r2指标:', r2_score(y_test, pxy))
print('ev指标:', explained_variance_score(y_test, pxy))
end = time.time()
print('time:', end-start)

MAE指标: 0.5923504185073079
MSE指标: 0.5842110479024896
RMSE指标: 0.7643369989098326
r2指标: 0.9210565716676742
ev指标: 0.9299811005363616
time: 0.09030818939208984


In [42]:
start = time.time()
# 02数据预测计算指标
pxy = dnn.predict(X02.reshape(20, 100))
pxy = np.squeeze(pxy)
print('MAE指标:', mean_absolute_error(y02, pxy))
print('MSE指标:', mean_squared_error(y02, pxy))
print('RMSE指标:', math.sqrt(mean_squared_error(y02, pxy)))
print('r2指标:', r2_score(y02, pxy))
print('ev指标:', explained_variance_score(y02, pxy))
end = time.time()
print('time:', end-start)

MAE指标: 0.4972196419835091
MSE指标: 0.4285942016590419
RMSE指标: 0.6546710637098924
r2指标: 0.9472487192311967
ev指标: 0.9538703052119051
time: 0.04231882095336914


In [43]:
res.loc['DNN','MAE'] = mean_absolute_error(y02, pxy)
res.loc['DNN','RMSE'] = math.sqrt(mean_squared_error(y02, pxy))
res.loc['DNN','R2'] = r2_score(y02, pxy)
res.loc['DNN','TIME'] = end-start

## 导入GRU模型计算指标

In [44]:
# 加载预训练模型
gru = load_model('./models/GRU_2021.h5')
gru.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru (GRU)                    (None, 100)               30900     
_________________________________________________________________
dense (Dense)                (None, 2)                 202       
Total params: 31,102
Trainable params: 31,102
Non-trainable params: 0
_________________________________________________________________


In [45]:
start = time.time()
# 预测测试集计算指标
pxy = gru.predict(X_test.reshape(400, 100, 1))
pxy = np.squeeze(pxy)
print('MAE指标:', mean_absolute_error(y_test, pxy))
print('MSE指标:', mean_squared_error(y_test, pxy))
print('RMSE指标:', math.sqrt(mean_squared_error(y_test, pxy)))
print('r2指标:', r2_score(y_test, pxy))
print('ev指标:', explained_variance_score(y_test, pxy))
end = time.time()
print('time:', end-start)

MAE指标: 0.2756353968404236
MSE指标: 0.15056168751058732
RMSE指标: 0.38802279251428945
r2指标: 0.9791365168960855
ev指标: 0.9791504838038096
time: 0.4181098937988281


In [46]:
start = time.time()
# 02数据预测计算指标
pxy = gru.predict(X02.reshape(20, 100, 1))
pxy = np.squeeze(pxy)
print('MAE指标:', mean_absolute_error(y02, pxy))
print('MSE指标:', mean_squared_error(y02, pxy))
print('RMSE指标:', math.sqrt(mean_squared_error(y02, pxy)))
print('r2指标:', r2_score(y02, pxy))
print('ev指标:', explained_variance_score(y02, pxy))
end = time.time()
print('time:', end-start)

MAE指标: 0.18800129586458203
MSE指标: 0.06782028395047054
RMSE指标: 0.2604232784343031
r2指标: 0.9917356271220192
ev指标: 0.991864596407389
time: 0.053143978118896484


In [47]:
res.loc['GRU','MAE'] = mean_absolute_error(y02, pxy)
res.loc['GRU','RMSE'] = math.sqrt(mean_squared_error(y02, pxy))
res.loc['GRU','R2'] = r2_score(y02, pxy)
res.loc['GRU','TIME'] = end-start

## 导入B_GRU模型计算指标

In [48]:
# 加载预训练模型
bgru = load_model('./models/B_GRU_2021.h5')
bgru.summary()

Model: "Model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 100, 1)]     0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, 100, 1)]     0                                            
__________________________________________________________________________________________________
gru_2 (GRU)                     (None, 100)          30900       input_3[0][0]                    
__________________________________________________________________________________________________
gru_3 (GRU)                     (None, 100)          30900       input_4[0][0]                    
______________________________________________________________________________________________

In [49]:
start = time.time()
# 预测测试集计算指标
pxy = bgru.predict([X_test, X_test_2])
pxy = np.squeeze(pxy)
print('MAE指标:', mean_absolute_error(y_test, pxy))
print('MSE指标:', mean_squared_error(y_test, pxy))
print('RMSE指标:', math.sqrt(mean_squared_error(y_test, pxy)))
print('r2指标:', r2_score(y_test, pxy))
print('ev指标:', explained_variance_score(y_test, pxy))
end = time.time()
print('time:', end-start)

MAE指标: 0.29613184619713584
MSE指标: 0.17426436188599503
RMSE指标: 0.41744983157979
r2指标: 0.9759176550884424
ev指标: 0.9769253439527762
time: 0.8249709606170654


In [50]:
start = time.time()
# 02数据预测计算指标
pxy = bgru.predict([X02, X02_2])
pxy = np.squeeze(pxy)
print('MAE指标:', mean_absolute_error(y02, pxy))
print('MSE指标:', mean_squared_error(y02, pxy))
print('RMSE指标:', math.sqrt(mean_squared_error(y02, pxy)))
print('r2指标:', r2_score(y02, pxy))
print('ev指标:', explained_variance_score(y02, pxy))
end = time.time()
print('time:', end-start)

MAE指标: 0.26058854961395267
MSE指标: 0.11675852761181657
RMSE指标: 0.3416994697271516
r2指标: 0.9857048121084138
ev指标: 0.9881319972495695
time: 0.05066227912902832


In [51]:
res.loc['B_GRU','MAE'] = mean_absolute_error(y02, pxy)
res.loc['B_GRU','RMSE'] = math.sqrt(mean_squared_error(y02, pxy))
res.loc['B_GRU','R2'] = r2_score(y02, pxy)
res.loc['B_GRU','TIME'] = end-start

## 查看统计结果并保存

In [52]:
res

,MAE,RMSE,R2,TIME
RNN,1.775772,2.138555,0.436643,0.052778
DNN,0.497220,0.654671,0.947249,0.042319
SVM,0.372712,0.459728,0.974035,0.018317
LSTM,0.241644,0.339661,0.985914,0.063001
GRU,0.188001,0.260423,0.991736,0.053144
B_GRU,0.260589,0.341699,0.985705,0.050662


In [53]:
res.to_csv('./doc/indictor.csv')